In [1]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

In [2]:
import os
import json

In [3]:
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [5]:
chat_service = skaoai.AzureChatCompletion("Your Azure OpenAI Service gpt-4-32k Model Delployment Name",endpoint,api_key=api_key,api_version = "2023-12-01-preview")
embedding_gen = skaoai.AzureTextEmbedding("Your Azure OpenAI Service text-embedding-ada-002 Model Delployment Name", endpoint,api_key=api_key,api_version = "2023-12-01-preview")

In [6]:
kernel.add_service(chat_service)


kernel.add_service(embedding_gen)

In [7]:
base_plugin = "./plugins"

In [8]:
files_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "FilePlugin")

In [9]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [10]:
kblist = []

In [11]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = await kernel.invoke(notesFunc, sk.KernelArguments(input=content))
    json_result = json.loads(str(result))
    kblist.append(json_result)
    file.close()

Error parsing XML of prompt: not well-formed (invalid token): line 3, column 50
Error parsing XML of prompt: not well-formed (invalid token): line 3, column 50
Error parsing XML of prompt: not well-formed (invalid token): line 3, column 50


In [12]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = await kernel.invoke(transcripsFunc, sk.KernelArguments(input=content))
    # print(result.result)
    json_result = json.loads(str(result))
    for item in json_result:
        kblist.append(item)
    file.close()

In [13]:
kblist

[{'kb': 'History of machine learning',
  'content': "The history of machine learning and artificial intelligence is marked by significant discoveries and periods of progress and stagnation. Notable discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). Key periods include the conceptualization of 'machines that can think' by Alan Turing in 1950, the Dartmouth Summer Research Project in 1956, the 'golden years' of AI from 1956 to 1974, the 'AI Winter' from 1974 to 1980 due to limitations and criticisms, the rise of 'expert systems' in the 1980s, the 'AI Chill' from 1987 to 1993 due to over-specialization, and the resurgence of ML and AI from 1993 to 2011 due to increased data and computational power. Today, ML and AI are integral parts of our lives, necessitating careful understanding of their impacts and ethical considerations."},
 {'kb': 

In [14]:
base_vectordb = 'aboutMLKBDemoDemo'

In [15]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

In [16]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection(base_vectordb)

In [17]:
memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=embedding_gen)

In [18]:
import uuid

In [19]:
uuid.uuid4()

UUID('6f25a53d-c468-455d-aae6-7863defd6736')

In [20]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await memory.save_information(base_vectordb, id=id, text=content)

In [21]:
ask = "can you tell me what is different ML and AI"

memories = await memory.search(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [22]:
result = ''
for item in memories:
    print(f"Top Result: {item.text} with score {item.relevance}")
    result = item.text

Top Result: The difference between AI and ML - AI is about machines accomplishing human-level tasks. Machine learning, a subset of AI, uses algorithms to learn from data. Deep learning is a subset of machine learning. with score 0.89043534


In [23]:
answer_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "AnswerPlugin")

In [24]:
answerFunc = answer_plugin["Summary"]

In [25]:
summary_result = await kernel.invoke(answerFunc, sk.KernelArguments(input=result))

In [26]:
str(summary_result)

'AI, or Artificial Intelligence, refers to machines performing tasks at a human level. Machine Learning, a part of AI, involves the use of algorithms that allow machines to learn from data. Deep Learning, a further subset of Machine Learning, is a more advanced form of learning. In simple terms, AI is the broad concept, Machine Learning is a way to achieve AI, and Deep Learning is a more complex method within Machine Learning.'